<a href="https://colab.research.google.com/github/srRyzhov/Hackathon/blob/main/NLP_SberBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install sentence-transformers

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
import numpy as np

import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sentence_transformers import SentenceTransformer
from transformers import BertForSequenceClassification, BertTokenizer


Загрузить файл CRA_train_1200.xlsx !

In [9]:
train_df = pd.read_excel('/content/CRA_train_1200.xlsx', engine = 'openpyxl', index_col = 0)

In [10]:
kriteriy_dict={'AAA': 7, 'AA': 6, 'A': 5, 'BBB': 4, 'BB': 3, 'B': 2, 'C': 1}
rating_dict={'AAA': 17, 'AA+': 16, 'AA': 15, 'AA-': 14, 'A+':13, 'A': 12, 'A-': 11, 'BBB+': 10, 'BBB': 9, 'BBB-': 8,'BB+': 7, 'BB': 6, 'BB-': 5, 'B+': 4,'B':3 ,'B-': 2, 'C': 1}

In [11]:
train_df

,pr_txt,Категория,Уровень рейтинга
Id,,,
1,Повышение кредитного рейтинга Акционерного об...,A,A
2,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB
3,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A
4,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA
5,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB
...,...,...,...
1207,«Эксперт РА» подтвердил рейтинг компании «Татн...,AAA,AAA
1208,«Эксперт РА» понизил рейтинг компании «Инвестг...,BB,BB
1209,АКРА подтвердило ПАО «КАМАЗ» кредитный рейтинг...,A,A+


In [12]:
def get_value(d, key):
    for k in d.keys():
        if k == key:
            return d[k]


In [13]:

train_df['Категория'] = train_df['Категория'].replace(kriteriy_dict)
train_df['Уровень рейтинга'] = train_df['Уровень рейтинга'].replace(rating_dict)
''' i = get_value(kriteriy_dict, i)
    train_df['Категория'][i] = get_value(kriteriy_dict, train_df['Категория'][i])
    train_df['Уровень рейтинга'][i] = get_value(rating_dict, train_df['Уровень рейтинга'][i])'''
train_df

,pr_txt,Категория,Уровень рейтинга
Id,,,
1,Повышение кредитного рейтинга Акционерного об...,5,12
2,«Эксперт РА» подтвердил кредитный рейтинг комп...,3,6
3,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",5,12
4,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,7,17
5,29 марта 2023 г. Ведущий рейтинговый аналитик ...,4,9
...,...,...,...
1207,«Эксперт РА» подтвердил рейтинг компании «Татн...,7,17
1208,«Эксперт РА» понизил рейтинг компании «Инвестг...,3,6
1209,АКРА подтвердило ПАО «КАМАЗ» кредитный рейтинг...,5,13


In [43]:
def cutting(release):
  c = 0
  for word in release:
    if word == '.':
      c+=1
  count = c - 7
  c = 0
  word_count = 0
  for word in release:
    word_count+=1
    if word == '.':
      c+=1
      if c==count:
        new_release = release[:word_count]
  return new_release




In [44]:
train_df['pr_txt'] = train_df['pr_txt'].apply(lambda x: cutting(x))

In [46]:
train_df['pr_txt'][2]

'«Эксперт РА» подтвердил кредитный рейтинг компании ООО «Энергоконцепции» на уровне <rating>  Москва, 31 марта  2023 г.  Рейтинговое агентство «Эксперт РА» подтвердило  рейтинг кредитоспособности  нефинансовой компании   ООО «Энергоконцепции»   на уровне <rating>. Прогноз по рейтингу – стабильный. ") ООО «Энергоконцепции» (далее – «компания») –  российская компания, занимающаяся поставкой, монтажом и обслуживанием  энергетического оборудования. Основные клиенты представлены производителями  тепловой и электрической энергии группы компаний ООО «Газпром энергохолдинг»  (<rating>, стаб.).   Агентство оценивает риск-профиль отрасли, в  которой компания ведет деятельность, нейтрально. Платежеспособность основных  клиентов компании находится на высоком уровне. Устойчивость к внешним шокам  оценивается агентством на среднем уровне, т.к. спрос на продукцию и услуги ООО  «Энергоконцепции» подвержен колебаниям экономических циклов. Барьеры для входа  находятся на умеренно низком уровне.   Рыночн

In [14]:
train_text = train_df['pr_txt'].astype('str').tolist()
train_labels7 = train_df['Категория'].tolist()
train_labels17 = train_df['Уровень рейтинга'].tolist()

In [15]:
# Загрузка предобученной модели Sentence-BERT на русском языке
model = SentenceTransformer('ai-forever/sbert_large_nlu_ru')


Тексты для классификации - train_text

Метки классов, соответствующие текстам - train_labels7

In [16]:
#Преобразование текстов в эмбеддинги
#embeddings = model.encode(train_text, convert_to_tensor=True)
embeddings = model.encode(train_text)

In [17]:
len(train_labels7)

1200

In [18]:
len(embeddings)

1200

In [20]:
'''tensor_cpu = torch.Tensor.cpu()
numpy_array = tensor_cpu.numpy()'''

'tensor_cpu = torch.Tensor.cpu()\nnumpy_array = tensor_cpu.numpy()'

In [22]:
# Разделение данных на обучающую и тестовую выборки
train_embeddings7, test_embeddings7, train_labels7, test_labels7 = train_test_split(embeddings, train_labels7, test_size=0.2, random_state=42,)


In [23]:
test_labels7

[6,
 4,
 4,
 5,
 3,
 7,
 5,
 6,
 5,
 5,
 7,
 4,
 6,
 5,
 3,
 5,
 6,
 6,
 4,
 5,
 5,
 7,
 5,
 2,
 4,
 1,
 7,
 4,
 4,
 4,
 5,
 4,
 3,
 5,
 6,
 4,
 5,
 7,
 5,
 4,
 5,
 4,
 7,
 6,
 7,
 5,
 6,
 7,
 6,
 5,
 5,
 5,
 4,
 4,
 6,
 6,
 3,
 5,
 4,
 5,
 2,
 7,
 3,
 6,
 4,
 6,
 7,
 7,
 5,
 7,
 4,
 4,
 6,
 5,
 7,
 7,
 5,
 3,
 6,
 7,
 3,
 5,
 5,
 7,
 4,
 6,
 7,
 3,
 7,
 5,
 2,
 5,
 5,
 6,
 4,
 5,
 5,
 4,
 3,
 5,
 2,
 7,
 4,
 2,
 4,
 6,
 4,
 7,
 5,
 3,
 1,
 4,
 6,
 5,
 3,
 6,
 5,
 2,
 5,
 4,
 5,
 7,
 5,
 6,
 5,
 5,
 5,
 5,
 1,
 3,
 5,
 4,
 7,
 6,
 4,
 6,
 5,
 7,
 6,
 5,
 7,
 2,
 5,
 7,
 3,
 7,
 7,
 3,
 4,
 2,
 4,
 3,
 4,
 7,
 4,
 5,
 5,
 5,
 4,
 2,
 6,
 6,
 5,
 5,
 4,
 4,
 6,
 6,
 4,
 6,
 5,
 6,
 4,
 5,
 5,
 6,
 4,
 4,
 5,
 5,
 4,
 7,
 5,
 3,
 6,
 4,
 4,
 5,
 2,
 4,
 3,
 7,
 7,
 5,
 3,
 5,
 3,
 3,
 4,
 5,
 7,
 7,
 3,
 7,
 4,
 4,
 4,
 6,
 3,
 4,
 3,
 7,
 5,
 7,
 5,
 5,
 4,
 2,
 5,
 4,
 6,
 7,
 5,
 3,
 3,
 7,
 3,
 4,
 1,
 6,
 2,
 7,
 6,
 5,
 7,
 5,
 4,
 7,
 6,
 5]

In [24]:
# Initialize and fit the scaler
scaler7 = StandardScaler()
train_embeddings_scaled7 = scaler7.fit_transform(train_embeddings7)
test_embeddings_scaled7 = scaler7.transform(test_embeddings7)

In [25]:
# Инициализация и обучение модели классификации

classifier7 = LogisticRegression(solver='liblinear')
#classifier7 = LogisticRegression(max_iter=1000, C=0.1)
#classifier7 = LogisticRegression(max_iter=1000, solver='sag')
# можно попробовать 'sag', 'saga' или 'newton-cg':

classifier7.fit(train_embeddings7, train_labels7)
#classifier.fit(train_embeddings.cpu(), train_labels)

LogisticRegression(solver='liblinear')

In [26]:
# Предсказание на тестовой выборке

#predictions = classifier7.predict(test_embeddings.cpu())
predictions7 = classifier7.predict(test_embeddings7)

# Оценка точности классификатора

accuracy7 = (predictions7 == test_labels7).mean()
print("Accuracy:", accuracy7)



Accuracy: 0.6208333333333333


In [27]:
# Отчет о классификации

report7 = classification_report(test_labels7, predictions7)
print("Classification Report:")
print(report7)

Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.75      0.86         4
           2       0.60      0.25      0.35        12
           3       0.44      0.31      0.36        26
           4       0.55      0.67      0.60        52
           5       0.59      0.69      0.64        68
           6       0.66      0.51      0.58        37
           7       0.83      0.83      0.83        41

    accuracy                           0.62       240
   macro avg       0.67      0.57      0.60       240
weighted avg       0.62      0.62      0.61       240



In [28]:
#Вывод ответов

answers7 = pd.DataFrame({'Id': 0, 'answer': test_labels7, 'prediction':predictions7 })
answers7

,Id,answer,prediction
0,0,6,6
1,0,4,4
2,0,4,5
3,0,5,5
4,0,3,4
...,...,...,...
235,0,5,5
236,0,4,4
237,0,7,7
238,0,6,7


In [29]:
#Преобразование текстов в эмбеддинги
#embeddings = model.encode(train_text, convert_to_tensor=True)
embeddings17 = model.encode(train_text)

In [30]:
#Для детализированной оценки
# Разделение данных на обучающую и тестовую выборки
#Поменял test_size с 0.2 на 0.1, т.к. сложнее выход
train_embeddings17, test_embeddings17, train_labels17, test_labels17 = train_test_split(embeddings, train_labels17, test_size=0.2, random_state=42)


In [31]:
len(train_labels17)

960

In [32]:
len(embeddings)

1200

In [36]:
# Initialize and fit the scaler
scaler17 = StandardScaler()
train_embeddings_scaled17 = scaler17.fit_transform(train_embeddings17)
test_embeddings_scaled17 = scaler17.transform(test_embeddings17)

In [37]:
# Инициализация и обучение модели классификации

classifier17 = LogisticRegression(solver='liblinear')
#classifier17 = LogisticRegression(max_iter=1000, C=0.1)
#classifier17 = LogisticRegression(max_iter=1000, solver='sag')
# можно попробовать 'sag', 'saga' или 'newton-cg':

classifier17.fit(train_embeddings17, train_labels17)
#classifier.fit(train_embeddings.cpu(), train_labels)

LogisticRegression(solver='liblinear')

In [38]:
# Предсказание на тестовой выборке

#predictions17 = classifier17.predict(test_embeddings.cpu())
predictions17 = classifier17.predict(test_embeddings17)

# Оценка точности классификатора

accuracy17 = (predictions17 == test_labels17).mean()
print("Accuracy:", accuracy17)



Accuracy: 0.4375


In [39]:
# Отчет о классификации

report17 = classification_report(test_labels17, predictions17)
print("Classification Report:")
print(report17)

Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.75      0.86         4
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3
           4       0.67      0.29      0.40         7
           5       0.00      0.00      0.00         5
           6       0.33      0.38      0.35         8
           7       0.33      0.08      0.12        13
           8       0.62      0.28      0.38        18
           9       0.35      0.32      0.33        19
          10       0.16      0.33      0.22        15
          11       0.31      0.31      0.31        32
          12       0.29      0.24      0.26        17
          13       0.44      0.74      0.55        19
          14       0.75      0.27      0.40        11
          15       0.54      0.44      0.48        16
          16       0.36      0.50      0.42        10
          17       0.74      0.90      0.81        41

   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
#Вывод ответов

answers17 = pd.DataFrame({'answer': test_labels17, 'prediction':predictions17 })
answers17

,answer,prediction
0,16,15
1,10,10
2,10,10
3,11,11
4,7,9
...,...,...
235,12,13
236,8,9
237,17,17
238,15,17


KeyError: ignored